# Atwater Customer Recommendations

#### A demo using DataStax Enterprise Analytics, Apache Cassandra, Apache Spark, python and Jupyter Notebooks to utilize the power of big customer data to recommend items to our customers with a high degree of accruacy 

### Things To Setup
* Install DSE https://docs.datastax.com/en/install/doc/install60/installTOC.html
* Start DSE Analytics Cluster
* Using Python 2.7
* Using DSE Analytics 6
* Using latest verion of Jupyter 
* Find full path to <>/lib/pyspark.zip
* Find full path to <>/lib/py4j-0.10.4-src.zip
* Start Jupyter with DSE to get all environemnt variables: dse exec jupyter notebook
* Make sure that the two CSV files are in the same locations as this notebook
* !pip install cassandra-driver
* !pip install pattern 
* !pip install panadas
* Counter-intuitive don't install pyspark!!

#### Add some environment variables to find dse verision of pyspark. Edit these varibles with your path.

In [29]:
pysparkzip = "/usr/share/dse/spark/python/lib/pyspark.zip"
py4jzip = "/usr/share/dse/spark/python/lib/py4j-0.10.4-src.zip"

In [30]:
# Needed to be able to find pyspark libaries
import sys
sys.path.append(pysparkzip)
sys.path.append(py4jzip)

#### Import python packages -- all are required

In [33]:
import pandas
import cassandra
import pyspark
import re
import os
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pattern.en import sentiment, positive

#### Helper function to have nicer formatting of Spark DataFrames

In [34]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  5, truncate = True):
    if(truncate):
        pandas.set_option('display.max_colwidth', 50)
    else:
        pandas.set_option('display.max_colwidth', -1)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

### Creating Tables, Pulling Tweets, and Loading Tables

#### Connect to DSE Analytics Cluster

In [35]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1']) #If you have a locally installed DSE cluster
session = cluster.connect()

#### Create Demo Keyspace 

In [36]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS demo 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set keyspace 

In [37]:
session.set_keyspace('demo')

#### Create the customer transaction table in DSE. Our primary key will be on customer id and state, and our clustering columns will be around gender, age, and rewards member status. Consider your data model when choosing your primary key. 

In [84]:
query = "CREATE TABLE IF NOT EXISTS customer_transactions (id int, \
                                                            customer_name text, \
                                                            gender text, age int, \
                                                            state text, home_store int, \
                                                            items list<text>, year int, \
                                                            month int, rewards_member text, \
                                                            PRIMARY KEY ((id, state), gender, age))"
session.execute(query)


#### Loading Testing Data from CSV file

In [85]:
query = "CREATE TABLE IF NOT EXISTS customer_test (id int, \
                                                            customer_name text, \
                                                            gender text, age int, \
                                                            state text, home_store int, \
                                                            items list<text>, year int, \
                                                            month int, rewards_member text, \
                                                            PRIMARY KEY ((id, state), gender, age))"
session.execute(query)

In [96]:
query = "CREATE TABLE IF NOT EXISTS customer_recommend (id int, \
                                                            customer_name text, \
                                                            gender text, age int, \
                                                            state text, home_store int, \
                                                            items list<text>, year int, \
                                                            month int, rewards_member text,\
                                                            prediction list<text>,\
                                                            PRIMARY KEY ((id, state), gender, age))"
session.execute(query)

In [86]:
!cqlsh -f loadCustomer.cql
!cqlsh -f loadCustomerTest.cql

Using 3 child processes

Starting copy of demo.customer_transactions with columns [id, customer_name, gender, age, state, home_store, items, year, month, rewards_member].
loadCustomer.cql:2:Failed to import 1 rows: ParseError - Invalid row length 0 should be 10,  given up without retries
loadCustomer.cql:2:Failed to process 1 rows; failed rows written to import_demo_customer_transactions.err
Processed: 3 rows; Rate:       5 rows/s; Avg. rate:       7 rows/s
3 rows imported from 1 files in 0.426 seconds (0 skipped).
Using 3 child processes

Starting copy of demo.customer_test with columns [id, customer_name, gender, age, state, home_store, items, year, month, rewards_member].
Processed: 1 rows; Rate:       3 rows/s; Avg. rate:       3 rows/sProcessed: 1 rows; Rate:       2 rows/s; Avg. rate:       3 rows/s
1 rows imported from 1 files in 0.389 seconds (0 skipped).


#### Do a select * on each table and verify that the tweets have been inserted into each Cassandra table

In [87]:
query = 'SELECT * FROM customer_transactions limit 10'
rows = session.execute(query)
for user_row in rows:
    print (user_row.id, user_row.items)

(2, [u'Collar', u'Bed', u'Bowl'])
(1, [u'Collar', u'Sweater', u'Bed'])


### Finally time for Apache Spark! 

#### Create a spark session that is connected to Cassandra. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [89]:
spark = SparkSession.builder.appName('demo').master("local").getOrCreate()

tableDF = spark.read.format("org.apache.spark.sql.cassandra").options(table="customer_transactions", keyspace="demo").load()

testDF = spark.read.format("org.apache.spark.sql.cassandra").options(table="customer_test", keyspace="demo").load()


print "Table Train Count: "
print tableDF.count()
showDF(tableDF)

print "Table Test Count: "
print testDF.count()
showDF(testDF)

Table Train Count: 
2


,id,state,gender,age,customer_name,home_store,items,month,rewards_member,year
0,2,CA,M,7,Rocky Bucaojit,21,"[Collar, Bed, Bowl]",11,N,2014
1,1,CA,M,14,Toby Moran,20,"[Collar, Sweater, Bed]",12,N,2014


Table Test Count: 
1


,id,state,gender,age,customer_name,home_store,items,month,rewards_member,year
0,3,CA,M,7,Max Moran,24,[Bed],2,Y,2015


#### Use FPGrowth to find Recommendation

In [93]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(tableDF)
recommendDF=model.transform(testDF)
recommendDF.show()

+---+-----+------+---+-------------+----------+-----+-----+--------------+----+----------+
| id|state|gender|age|customer_name|home_store|items|month|rewards_member|year|prediction|
+---+-----+------+---+-------------+----------+-----+-----+--------------+----+----------+
|  3|   CA|     M|  7|    Max Moran|        24|[Bed]|    2|             Y|2015|  [Collar]|
+---+-----+------+---+-------------+----------+-----+-----+--------------+----+----------+



In [97]:
recommendDF.write.format("org.apache.spark.sql.cassandra").options(table="customer_recommend", keyspace="demo").save()

In [99]:
query = 'SELECT * FROM customer_recommend limit 10'
rows = session.execute(query)
for user_row in rows:
    print (user_row.id, user_row.items, user_row.prediction)

(3, [u'Bed'], [u'Collar'])
